<a href="https://colab.research.google.com/github/phamkhoa18/Backend_Bongdadientu/blob/main/Code_ho%C3%A0n_t%E1%BA%A5t_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. CÀI ĐẶT VÀ IMPORT
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

from unsloth import FastLanguageModel
import torch

  Resolved https://github.com/unslothai/unsloth.git to commit bcb1bcfd6aa9dc7db8910c733cecf0c676b37eb3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.8/184.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 17.5 MB/s eta 0:00:00
  Created wheel for unsloth: filename=unsloth-2025.8.9-py3-none-any.whl size=312437 sha256=58c56e78e51c139e8216aa96b60844362cea420a2f83548b57277b0b80697cd4
  Stored in directory: /tmp/pip-ephem-wheel-cache-22psxccr/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5

In [ ]:
# 2. LOAD MODEL CHO LUẬT VIỆT NAM
max_seq_length = 2048  # Tăng lên để phù hợp với câu hỏi luật phức tạp
dtype = None  # Auto detection
load_in_4bit = True  # Tiết kiệm memory

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",  # Llama-3-8B tốt cho tiếng Việt
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
# 3. SETUP PEFT CHO FINETUNE
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,  # Tăng từ 16 lên 64 cho domain phức tạp như luật
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Tăng learning rate
    lora_dropout = 0.05,  # Giảm dropout vì luật cần precision
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.8.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
# 4. PROMPT TEMPLATE CHO CHATBOT LUẬT VIỆT NAM
law_prompt = """Bạn là một trợ lý AI chuyên về luật pháp Việt Nam. Hãy trả lời câu hỏi một cách chính xác, dựa trên các điều luật hiện hành.

### Câu hỏi:
{}

### Trả lời:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["question"]  # Cột câu hỏi luật
    outputs = examples["answer"]        # Cột câu trả lời
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = law_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [ ]:
from datasets import load_dataset

# Load dataset từ Hugging Face
dataset = load_dataset("iamnguyen/qa_luat", split="train")
print(f"Dataset có {len(dataset)} mẫu")
print("Cấu trúc dataset:")
print(dataset.features)
print("\nVí dụ một mẫu:")
print(dataset[0])

README.md:   0%|          | 0.00/421 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/6.32M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6409 [00:00<?, ? examples/s]

Dataset có 6409 mẫu
Cấu trúc dataset:
{'date': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'citation': Value(dtype='string', id=None), 'answer': Value(dtype='string', id=None), 'refs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

Ví dụ một mẫu:
{'date': '28/03/2024', 'question': 'Thanh niên có việc làm ổn định có được miễn nghĩa vụ quân sự?', 'citation': 'Điều 41. Tạm hoãn gọi nhập ngũ và miễn gọi nhập ngũ\n...\n2. Miễn gọi nhập ngũ đối với những công dân sau đây:\na) Con của liệt sĩ, con của thương binh hạng một;\nb) Một anh hoặc một em trai của liệt sĩ;\nc) Một con của thương binh hạng hai; một con của bệnh binh suy giảm khả năng lao động từ 81% trở lên; một con của người nhiễm chất độc da cam suy giảm khả năng lao động từ 81 % trở lên;\nd) Người làm công tác cơ yếu không phải là quân nhân, Công an nhân dân;\nđ) Cán bộ, công chức, viên chức, thanh niên xung phong được điều động đến công tác, làm việc ở vùng có điều kiện kinh t

In [ ]:
# Apply formatting cho dataset thực
dataset = dataset.map(formatting_prompts_func, batched=True)
print("\nDataset sau khi format:")
print(dataset[0]['text'][:500] + "...")

Map:   0%|          | 0/6409 [00:00<?, ? examples/s]


Dataset sau khi format:
Bạn là một trợ lý AI chuyên về luật pháp Việt Nam. Hãy trả lời câu hỏi một cách chính xác, dựa trên các điều luật hiện hành.

### Câu hỏi:
Thanh niên có việc làm ổn định có được miễn nghĩa vụ quân sự?

### Trả lời:
Căn cứ tại khoản 2 Điều 41 Luật Nghĩa vụ quân sự 2015 có quy định về miễn thực hiện nghĩa vụ quân sự như sau:

Điều 41. Tạm hoãn gọi nhập ngũ và miễn gọi nhập ngũ
...
2. Miễn gọi nhập ngũ đối với những công dân sau đây:
a) Con của liệt sĩ, con của thương binh hạng một;
b) Một anh hoặc...


In [ ]:
# Chỉ lấy 50 mẫu đầu để test
dataset_small = dataset.select(range(200))
print(f"Sử dụng {len(dataset_small)} mẫu cho training")

# Apply formatting
dataset_small = dataset_small.map(formatting_prompts_func, batched=True)
print("Dataset sau khi format:")
print(dataset_small[0]['text'][:500] + "...")

Sử dụng 200 mẫu cho training


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset sau khi format:
Bạn là một trợ lý AI chuyên về luật pháp Việt Nam. Hãy trả lời câu hỏi một cách chính xác, dựa trên các điều luật hiện hành.

### Câu hỏi:
Thanh niên có việc làm ổn định có được miễn nghĩa vụ quân sự?

### Trả lời:
Căn cứ tại khoản 2 Điều 41 Luật Nghĩa vụ quân sự 2015 có quy định về miễn thực hiện nghĩa vụ quân sự như sau:

Điều 41. Tạm hoãn gọi nhập ngũ và miễn gọi nhập ngũ
...
2. Miễn gọi nhập ngũ đối với những công dân sau đây:
a) Con của liệt sĩ, con của thương binh hạng một;
b) Một anh hoặc...


In [ ]:
# 6. TRAINING ARGUMENTS CHO LUẬT - OPTIMIZED FOR COLAB 15GB
from trl import SFTTrainer, SFTConfig
import os
import gc
import torch

# Tắt wandb và tối ưu memory
os.environ["WANDB_DISABLED"] = "true"
torch.cuda.empty_cache()
gc.collect()

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_small,  # Chỉ 50 samples
    dataset_text_field = "text",
    max_seq_length = 512,  # Giảm từ 2048 xuống 512
    dataset_num_proc = 1,  # Giảm từ 2 xuống 1
    packing = True,  # Bật packing để tiết kiệm memory
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,  # Tăng để bù batch size nhỏ
        warmup_steps = 5,  # Giảm warmup
        max_steps = 20,  # Giới hạn steps thay vì epochs
        learning_rate = 5e-4,  # Tăng learning rate do ít steps
        fp16 = True,  # Force fp16 để tiết kiệm memory
        bf16 = False,
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs_law_chatbot",
        save_strategy = "no",  # Không save checkpoint
        dataloader_pin_memory = False,  # Tắt pin memory
        dataloader_num_workers = 0,  # Tắt multiprocessing
        remove_unused_columns = True,
        report_to = [],
        gradient_checkpointing = True,  # Tiết kiệm memory
    ),
)

# Memory cleanup trước khi train
torch.cuda.empty_cache()
gc.collect()

# 7. BẮT ĐẦU TRAINING
trainer_stats = trainer.train()

# Cleanup sau khi train
torch.cuda.empty_cache()
gc.collect()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 1 | Total steps = 20
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,1.218800
10,0.770200
15,0.663000
20,0.679900


3225

In [ ]:
# 8. TEST MODEL SAU KHI TRAIN
from transformers import TextStreamer

# Chuyển model sang inference mode
FastLanguageModel.for_inference(model)

def test_law_chatbot(question):
    """Test chatbot với câu hỏi luật"""
    messages = [
        {'role': 'system', 'content': 'Bạn là trợ lý AI chuyên về luật pháp Việt Nam. Hãy trả lời chính xác dựa trên các quy định pháp luật.'},
        {'role': 'user', 'content': question}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # Generate với streaming để xem real-time
    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        inputs,
        max_new_tokens=256,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer,
        use_cache=True
    )

    # Trả về text đầy đủ nếu cần
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full_response.split("assistant")[-1].strip()

# Test với các câu hỏi mẫu

test_questions = [
    "Điều kiện kết hôn của người đang thực hiện nghĩa vụ quân sự là gì?"
]

print("=== TESTING VIETNAM LAW CHATBOT ===\n")

for i, question in enumerate(test_questions, 1):
    print(f"Câu hỏi {i}: {question}")
    print("Trả lời:")
    response = test_law_chatbot(question)
    print("-" * 60 + "\n")

# Test interactive
print("=== INTERACTIVE TEST ===")
while True:
    user_question = input("Nhập câu hỏi luật (hoặc 'quit' để thoát): ")
    if user_question.lower() == 'quit':
        break

    print("Chatbot:")
    test_law_chatbot(user_question)
    print("\n")

=== TESTING VIETNAM LAW CHATBOT ===

Câu hỏi 1: Điều kiện kết hôn của người đang thực hiện nghĩa vụ quân sự là gì?
Trả lời:
Căn cứ Điều 10 Luật Hôn nhân và gia đình 2014 quy định về điều kiện kết hôn như sau:

Điều kiện kết hôn
1. Người kết hôn phải đủ 20 tuổi trở lên.
2. Người kết hôn phải có năng lực hành vi dân sự.
3. Người kết hôn phải được sự đồng ý của cha mẹ hoặc người giám hộ hợp pháp nếu chưa đủ 21 tuổi.
4. Người kết hôn không phải là anh, chị, em ruột của nhau.
5. Người kết hôn không phải là cha, mẹ hoặc con của nhau.
6. Người kết hôn không phải là vợ, chồng của nhau trong thời gian đã kết hôn.
7. Người kết hôn không phải là người đang bị truy cứu trách nhiệm hình sự, đang chấp hành hình phạt tù, đang bị áp dụng biện pháp giáo dục tại xã, phường, thị trấn hoặc đang bị áp dụng biện pháp đưa vào trường giáo dưỡng.
8. Người kết hôn không phải là người bị mất năng lực hành vi dân sự hoặc đang bị hạn chế năng lực hành vi dân sự.
9. Người kết hôn không phải là người đang bị áp dụng

In [ ]:
model.save_pretrained("vietnam_law_chatbot")
tokenizer.save_pretrained("vietnam_law_chatbot")

('vietnam_law_chatbot/tokenizer_config.json',
 'vietnam_law_chatbot/special_tokens_map.json',
 'vietnam_law_chatbot/chat_template.jinja',
 'vietnam_law_chatbot/tokenizer.json')

In [ ]:
import shutil
from google.colab import files

# Nén thư mục thành file zip
shutil.make_archive("/content/vietnam_law_chatbot", 'zip', "/content/vietnam_law_chatbot")

# Tải file zip về máy
files.download("/content/vietnam_law_chatbot.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>